# **Pollution Analysis** (Flume Videos)

This is a continuation of our analysis notebook from yesterday! Here are the differences:

* We are setting up our code in such a way that we can save concentrations from multiple videos to the same DataFrame
* We are saving our DataFrame to a CSV file for later use in another notebook

## Step 1: Install the `pyvideoreader` package

This package contains `videoreader`. From `videoreader`, we will import `VideoReader` to read in video data.

In [ ]:
#Install the required package for reading in video data
!pip install pyvideoreader

## Step 2: Import the other packages we will use

*   `numpy`
*   `matplotlib.pyplot`
*   `files` (for uploading files to your Colab workspace)
*   `VideoReader` (for reading data from video frames)
*   `plotly.express` (for creating interactive, inline figures)
*   `cv2` (for saving frame data to image files)
*   `pandas` (for collecting and handling our data before plotting)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from videoreader import VideoReader
import plotly.express as px
import cv2
import pandas as pd

## Step 3: Download flume videos from Canvas to your computer

1.   Go to this week's site on Canvas (Create with Code - Environmental Modeling)
2.   Click on 'Modules' from the menu on the left-hand side
3.   Scroll down to the 'Data' section
4.   Click on the video you want to work with
5.   Click the Download link


## Step 4: Upload the video to Colab

1.   Run the cell below
2.   Click on `'Choose Files'`
3.   Select the video file from your computer
4.   Wait for the video to upload to your workspace

NOTE: If you close this tab, you will have to upload the video again. However, if you restart the kernel, the video will still be in your workspace. You can skip this cell once you have already uploaded the video or press `'Cancel'` when asked to upload a video after running it.

In [ ]:
# This command allows you to upload a file from your computer to the cloud where this notebook exists
files.upload()

## Step 5: Read in the video file with `VideoReader` 

We use VideoReader to read in frames from our video. After doing so, we will print the following information:

*   video file name
*   number of frames
*   size of the frames
*   frame rate

In [ ]:
# !!! Insert the name of the video you are uploading below BEFORE uploading the file to avoid errors. !!! 
videoName = 'NoObs_0p1mpers_short.mp4'
video_file = VideoReader(videoName)

print(video_file)  # Prints video file name, number of frames, frame size, and frame rate

The size `(720, 1280, 3)` tells us that each frame has a height of `720` pixels and width of `1280` pixels. The `3` tells us that each image has three channels; these are the color channels! The first one is **red**, the second is **green**, and the third is **blue** (Have you heard of RGB color values?).

The frame rate of `30.00 fps` tells us that thirty frames are taken every second (`fps` = frames per second).

## Step 6: Convert the video file to individual image files

We will use methods from `cv2` to read a video file, then loop through each frame in the video and save it as a JPEG image file. This allows us to step through time, in a sense, and analyze the video frames independently of each other.

The cell below is set up to load 10 frames (`end = 10`). To load a different number of frames, change `end =` to another number. As a reminder, you cannot load in more frames than the number that exist in the video. You can find the number of available frames by looking at the output from the last cell (our video has 54 frames in total).

In [ ]:
file_count = 0
start = 1
end = 5 # Replace this number with however many frames you want to work with

frame_count = 0
cap2 = cv2.VideoCapture(videoName)
# While the video file is open, we iterate through the video, extract the frames, and save them to a JPEG image file 
while (cap2.isOpened()):
    if frame_count >= start and frame_count < (end+1):
        success,image = cap2.read()
        if success == False:
            break
        cv2.imwrite('Frame_'+str(frame_count)+'.jpg', image) # save frame as JPEG file      
        print('Read a new frame: ', success)
    frame_count += 1
    if frame_count == (end+1): # Close video file once frames have been retrieved
        cap2.release()

The printed values `True` above tell us that each frame was read successfully.

## Manipulating the images

Now that we have loaded in the frames we want to work with, we can start using them!

To get a better idea of what we are working with, let's output the first frame of our video:

In [ ]:
# Read the frame
img_rgb = plt.imread('Frame_1.jpg')

# Show the frame
fig = px.imshow(img_rgb)
fig.show()

In this image, we can see the plume, a tube of dye to the right, and a piece of paper toward the top of the image. Out of these, we are really **only interested in the plume**. So, let's crop the image to just look at the plume!

We don't want to include the tube of dye in our cropped image because it will skew our concentration calculations later on. Since we want to crop this out, we can choose a section of the image more toward the center.

Since the image file is read in as a matrix, we can use Python's **list slicing** capabilities to crop it. We can mouse over the image displayed in the previous cell's output to pick the indices we will use to crop the image.

Let's crop the vertical axis (rows) between `350` and `600`, and the horizonal axis (columns) between `300` and `1000`. The slicing syntax looks like:

```
cropped_image = image[350:600, 300:1000, :]
```

We pass `:` into the third entry because we want to keep all three color channels (red, green, blue).

I want to save these values to variables so it is easier to make changes later on. Setting `row_min = 350`, `row_max = 600`, `col_min = 300`, and `col_max = 1000` gives us the following syntax:

```
cropped_image = image[row_min:row_max, col_min:col_max, :]
```


In [ ]:
# Crop the image
row_min = 350
row_max = 600

col_min = 300
col_max = 1000

img_rgb_crop = img_rgb[row_min:row_max, col_min:col_max, :]

# Display the cropped image
fig = px.imshow(img_rgb_crop)
fig.show()

If the cropped image above looks good to you, then we can go ahead and crop the rest of the frames we are working with! If you want to make changes, do so before running the next cell by editing the `row_min`, `row_max`, `col_min`, and `col_max` values.

Each cropped image will get added to a `list` in a **for loop**. Then, the `list` will be converted to a `tuple` and passed into NumPy's `stack()` method. This method stacks the images in order; we do this so that we have one variable to work with rather than many.

In [ ]:
# Create an empty list to store the frames in
all_frames = []

for frame_count in range(end):  # 'end' contains the number of frames we chose to read in earlier
  all_frames.append(plt.imread('Frame_'+str(frame_count+1)+'.jpg')[row_min:row_max, col_min:col_max, :])

all_frames = tuple(all_frames)

img_rgb = np.stack(all_frames, axis = -1)

Lastly, let's **splice** our stack into three separate matrices, one for each **color channel**.

Red is stored in the first channel, green is stored in the second, and blue is stored in the third. Since Python begins indexing at `0`, this means red can be found at index `0`, green can be found at index `1`, and blue can be found at index `2` of the color channels.

In [ ]:
# Save the red channel to a new variable
Red = img_rgb[:,:,0,:]

# Save the green channel to a new variable
Green = img_rgb[:,:,1,:]

# Save the blue channel to a new variable
Blue = img_rgb[:,:,2,:]

With this step, we are done setting up our data and can move on to analysis!

## Splitting the plume into regions based on concentration

We are ready to **separate our images into regions** based on low, medium, and high **concentration**. We can think of these regions as light orange, medium orange, and dark orange, respectively, in the frame shown above. When the dye is **more dispersed**, it appears **lighter** and has a **lower concentration**. When the dye is **less dispersed**, it appears **darker** and has a **higher concentration**.

To pick color values that match low, medium, and high concentration, run your cursor over the plotted frame above. Make note of the RGB values that appear for light, medium, and dark regions.

We do not need to explicitly define medium values below -- anything that is between the light and dark color values will be placed in the 'medium' category.

We define some sample values here, but feel free to change them to match what you see in the frame you are working with:

In [ ]:
# Color (RGB) values for dark concentration
dark_red = 190 
dark_green = 90  
dark_blue = 40  

# Color (RGB) values for light concentration
light_red = 210
light_green = 125
light_blue = 50

We want to figure out which pixels in each cropped image fall into the light, medium, or dark concentration categories. Some pixels may not fall into any of them, and that is okay, too! It just means that those pixels are not counted as part of our plume. For example, pixels showing the background or floor of the flume likely won't be counted as part of our plume (and they shouldn't be, so this is good).

If the pixel in question fits into a light, medium, or dark category, we need to store this information. Let's first **preallocate** some space to do that:

In [ ]:
# Preallocating arrays that are the same size as our frames in the stack, but full of 0s
light = np.zeros(Red.shape)
med = np.zeros(Red.shape)
dark = np.zeros(Red.shape)

In [ ]:
print(light)

We see below that we have **nested loops** -- the outermost loop (`k`) goes through each of our frames, while the inner loops (`i` and `j`) go over the pixels in each image. `i` specifies the row number of a pixel, and `j` specifies the column number.

In the cell below, we decide which cateogory each pixel falls under.

In [ ]:
# Separate colors by dark, medium, and light color ranges for all three color channels

for k in range(np.size(Red,2)): # Loop over frames (time)
  for i in range(np.size(Red, 0)): # Loop over rows (vertical)
    for j in range(np.size(Red, 1)): # Loop over columns (horizontal)

      # Check whether pixel satisfies dark conditions
      if (Red[i,j,k] <= dark_red) and (Green[i,j,k] <= dark_green) and (Blue[i,j,k] <= dark_blue):
        dark[i,j,k] = 1
      
      # Check whether pixel satisfies medium conditions
      elif (dark_red < Red[i,j,k] < light_red) and (dark_green < Green[i,j,k] < light_green) and (dark_blue < Blue[i,j,k] < light_blue):
        med[i,j,k] = 1
      
      # Check whether pixel satisfies light conditions
      elif (Red[i,j,k] >= light_red) and (Green[i,j,k] >= light_green) and (Blue[i,j,k] >= light_blue):
        light[i,j,k] = 1

This leaves us with three matrices: `light`, `med`, and `dark`. These matrices have a `1` wherever a pixel fits their color/concentration requirements, and a `0` everywhere else. So, a pixel is only in the dark matrix if its red, green, *and* blue values are darker than the specified thresholds.


## It's time ... **finding the concentration!**

We just made some big changes to our data! To make sure we all understand the variables we are now working with, let's print out the dimensions:

In [ ]:
print('img_rgb:')
print('Number of rows per frame:', np.size(img_rgb,0))
print('Number of columns per frame:', np.size(img_rgb,1))
print('Number of channels per frame:', np.size(img_rgb,2))
print('Number of frames:', np.size(img_rgb,3))

print('\nlight:')
print('Number of rows per frame:', np.size(light,0))
print('Number of columns per frame:', np.size(light,1))
print('Number of frames:', np.size(light,2))

print('\nmed:')
print('Number of rows per frame:', np.size(med,0))
print('Number of columns per frame:', np.size(med,1))
print('Number of frames:', np.size(med,2))

print('\ndark:')
print('Number of rows per frame:', np.size(dark,0))
print('Number of columns per frame:', np.size(dark,1))
print('Number of frames:', np.size(dark,2))

Notice how the `light`, `med`, and `dark` matrices **no longer have three color channels**. This is because we do not need to store the color of a pixel -- we just need to know whether it fits in that category (`1`) or not (`0`).

Now, to find the concentration, we want to store the **number of light/medium/dark points in each frame**.

Just like how we preallocated storage space for `light`, `med`, and `dark`, we want to preallocated space for these numbers. Let's do that first:

In [ ]:
# Save the column and frame dimensions to variables so we don't have to recalculate several times
num_cols = np.size(img_rgb,1)
num_frames = np.size(img_rgb,3)

# Initialize arrays for storage
num_dark = np.zeros((num_cols, num_frames))
num_med = np.zeros((num_cols, num_frames))
num_light = np.zeros((num_cols, num_frames))

Now, we can find values to store in these preallocated spaces.

In [ ]:
# Loop over the columns
for i in range(num_cols): 
  # Loop over the frames 
  for j in range(num_frames):
    
    # Find the indices of nonzero pixels in the light, med, and dark matrices (places where pixels satisfy their conditions)
    # Previously, we saved pixels that satisfy their conditions as 1s, and everything else is 0
    # Becaues of this, we can look for all pixel with a value greater than 0 using NumPy's where() method
    dark_nonzero_idx = list(zip(*np.where(dark[:,:,j] > 0)))
    med_nonzero_idx = list(zip(*np.where(med[:,:,j] > 0)))
    light_nonzero_idx = list(zip(*np.where(light[:,:,j] > 0)))

    # To get the number of pixels with each concentration, we count how many indices were found with the len() method
    num_dark[i,j] = len(dark_nonzero_idx)
    num_med[i,j] = len(med_nonzero_idx)
    num_light[i,j] = len(light_nonzero_idx)

What we have now are three matrices that store the number of low/medium/high concentration pixels per column for all of our frames:

In [ ]:
print('\nnum_light:')
print('Number of entries per frame:', np.size(num_light,0))
print('Number of frames:', np.size(num_light,1))

print('\nnum_med:')
print('Number of entries per frame:', np.size(num_med,0))
print('Number of frames:', np.size(num_med,1))

print('\nnum_dark:')
print('Number of entries per frame:', np.size(num_dark,0))
print('Number of frames:', np.size(num_dark,1))

Lastly, let's calculate the **average** number of light/medium/dark pixels per frame. This gives us the low/medium/high concentration in every frame! To find the average, we use NumPy's `mean()` method.

In [ ]:
# Take the average of each concentration
light_mean = np.mean(num_light, axis = 0)
med_mean = np.mean(num_med, axis = 0)
dark_mean = np.mean(num_dark, axis = 0)

After calculating the averages, we divide each one by the maximum value (number of points) from that category's frames. To find the maximum, we use NumPy's `max()` method.

Doing this means we are **normalizing** the values; essentially, we are putting all three categories on the same scale so that we can make comparisons between them. After normalizing, all values should be between 0 and 1.


In [ ]:
# Normalize each mean
light_mean_norm = light_mean/np.max(num_light)
med_mean_norm = med_mean/np.max(num_med)
dark_mean_norm = dark_mean/np.max(num_dark)

## Visualizing Results

What we have now are three variables, `light_mean_norm`, `med_mean_norm`, and `dark_mean_norm`, that hold the **low, medium, and high concentrations for each frame**. For example, if we used 10 frames, then each of these variables is a NumPy array with 10 entries. The first entry has the concentration in the first frame, and so on.

In [ ]:
print('\nlight_mean_norm:')
print('Type:', type(light_mean_norm))
print('Number of entries:', np.size(light_mean_norm,0))

print('\nmed_mean_norm:')
print('Type:', type(med_mean_norm))
print('Number of entries:', np.size(med_mean_norm,0))

print('\ndark_mean_norm:')
print('Type:', type(dark_mean_norm))
print('Number of entries:', np.size(dark_mean_norm,0))

Let's **visualize** our results! To make plotting and referencing the data simpler, we can load our arrays into a **Pandas DataFrame** first. Recall that we first store the arrays in a dictionary, then pass the dictionary to the DataFrame constructor:

**The first time you make a DataFrame, run this cell:**

In [ ]:
# Creating an array with frame numbers
frames = np.linspace(1, end, num = len(light_mean_norm), axis = 0)

# Creating a dictionary with keys 'Low', 'Medium', and 'High' (referring to concentration levels), and 'Frame' (frame numbers)
# !!! Make sure you use correct key names !!!
data_to_plot = {'Frame': frames, 'Low_noObs_0p1': light_mean_norm, 'Medium_noObs_0p1': med_mean_norm, 'High_noObs_0p1': dark_mean_norm}

# Passing our dictionary to the DataFrame constructor
df = pd.DataFrame(data=data_to_plot)

df

**After the DataFrame has already been created, run this cell instead:**

In [ ]:
# Creating a dictionary with keys 'Low', 'Medium', and 'High' (referring to concentration levels), and 'Frame' (frame numbers)
# !!! Make sure you use correct key names !!!
data_to_plot = {'Frame': frames, 'Low_noObs_0p4': light_mean_norm, 'Medium_noObs_0p4': med_mean_norm, 'High_noObs_0p4': dark_mean_norm}

# Passing our dictionary to the DataFrame constructor
df2 = pd.DataFrame(data=data_to_plot)

df = df.merge(df2, how = 'inner', on = 'Frame')

df

Last step -- let's plot each of these!

In [ ]:
# Visualize the high concentration data
df.plot(kind = 'scatter', x = 'Frame', y = 'High', title = 'High concentrations over time', color = 'k')
plt.show()

In [ ]:
# Visualize the medium concentration data
df.plot(kind = 'scatter', x = 'Frame', y = 'Medium', title = 'Medium concentrations over time', color = 'k')
plt.show()

In [ ]:
# Visualize the low concentration data
df.plot(kind = 'scatter', x = 'Frame', y = 'Low', title = 'Low concentrations over time', color = 'k')
plt.show()

## Exporting your data to a CSV file:

In [ ]:
df.to_csv('flume_concentrations.csv', index = False)